# Day 5a: Agent2Agent Communication (LangChain可本地运行版)

🤝 **Agent2Agent (A2A) 通信**

欢迎来到 5 天 Agent 课程的第 5 天！

本 Notebook 将教你如何构建多 Agent 系统，使不同的 Agent 能够通过网络协议进行通信和协作。你将学习如何集成外部 Agent 服务，并在本地像使用普通工具一样使用远程 Agent。

在本 Notebook 中，你将：

- ✅ 理解 A2A (Agent2Agent) 协议的概念
- ✅ 使用 **LangServe** 将 Agent 暴露为 API 服务
- ✅ 使用 **RemoteRunnable** 调用远程 Agent
- ✅ 构建一个基于“产品目录 Agent”和“客户支持 Agent”的协作系统

---

**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。

## ⚙️ Section 0: Setup (环境设置)


### 0.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 0.2 安装依赖

本教程需要运行一个后台服务器来模拟远程 Agent，因此我们需要安装 `langserve` 及其依赖。

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community "langserve[all]" fastapi uvicorn sse-starlette


### 0.3 导入依赖库

In [1]:
import os
import json
import time
import subprocess
import requests
import warnings
from typing import Optional
from dotenv import load_dotenv, find_dotenv

# LangChain 核心
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.schema.runnable import RunnableLambda

# Remote Runnable (用于连接远程 Agent)
from langserve import RemoteRunnable

warnings.filterwarnings("ignore")
print("✅ 依赖库导入成功")

✅ 依赖库导入成功


### 0.4 配置 API Key

加载 `.env` 文件中的配置。

In [2]:
load_dotenv(find_dotenv())

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get("MODEL_NAME")
    api_key = os.environ.get("OPENAI_API_KEY")
    base_url = os.environ.get("OPENAI_API_BASE")
    tavily_key = os.environ.get("TAVILY_API_KEY")

    if not api_key:
        print("❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY")
        return None
    
    # 打印部分 Key 以验证 (隐藏敏感信息)
    masked_key = f"{api_key[:8]}..." if api_key else "None"
    print(f"✅ API Key 已加载: {masked_key}")
    print(f"✅ 使用模型: {model_name}")
    if base_url:
        print(f"✅ API Base URL: {base_url}")
    
    if tavily_key:
        print("✅ Tavily Search API 已启用 (增强搜索能力)")
    else:
        print("⚠️ 未找到 Tavily Key，将使用 DuckDuckGo 搜索 (免费但质量可能受限)")
    
    return {
        "model_name": model_name,
        "api_key": api_key,
        "base_url": base_url,
        "tavily_key": tavily_key,
        "extra_body": json.loads(os.environ.get("EXTRA_BODY", "{}"))
    }

config = configure_api_keys()

✅ API Key 已加载: ms-ec366...
✅ 使用模型: Qwen/Qwen3-235B-A22B-Instruct-2507
✅ API Base URL: https://api-inference.modelscope.cn/v1
✅ Tavily Search API 已启用 (增强搜索能力)


## 🏗️ A2A 架构概览

我们将构建两个 Agent：

1.  **Product Catalog Agent (服务端/供应商)**: 
    -   拥有查询产品详情的工具。
    -   运行在 `localhost:8001`。
    -   使用 **LangServe** 暴露服务。

2.  **Customer Support Agent (客户端/客服)**:
    -   负责回答客户问题。
    -   运行在当前 Notebook 中。
    -   拥有一个 "Remote Tool"，通过 API 调用 Product Catalog Agent。

这模拟了现实世界中微服务架构下的 Agent 协作：你无法直接通过代码调用外部服务，只能通过网络协议（如 HTTP）进行通信。

## 📦 Section 1: 创建 Product Catalog Agent (服务端)

首先，我们需要定义运行在服务端的 Agent。为了在 Notebook 中模拟这个过程，我们将服务端的代码写入一个独立的 Python 文件 `product_catalog_server.py`，稍后在后台运行它。

这个文件包含：
1. **Tools**: `get_product_info` 模拟查询数据库。
2. **Agent**: 一个标准的 LangChain Tool Calling Agent。
3. **Server**: 使用 `langserve.add_routes` 将 Agent 注册到 FastAPI 应用中。

In [3]:
server_code = '''
import os
import json
from dotenv import load_dotenv, find_dotenv
from fastapi import FastAPI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langserve import add_routes
import uvicorn
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda

# 1. 加载配置
load_dotenv(find_dotenv())
model_name = os.environ.get("MODEL_NAME", "gpt-3.5-turbo")
api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("OPENAI_API_BASE")
extra_body = json.loads(os.environ.get("EXTRA_BODY", "{}"))

# 2. 定义工具
@tool
def get_product_info(product_name: str) -> str:
    """获取指定产品的详细信息（价格、库存、规格）。"""
    print(f"DEBUG: get_product_info called with {product_name}")
    # 模拟数据库
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, 库存紧张 (8台), 128GB, 钛金属",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, 现货充足 (31台), 256GB, 幻影黑",
        "dell xps 15": 'Dell XPS 15, $1,299, 现货 (45台), 15.6寸显示屏, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14, $1,999, 现货 (22台), M3 Pro芯片, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 耳机, $399, 现货 (67台), 降噪, 30小时续航",
        "ipad air": 'iPad Air, $599, 现货 (28台), 10.9寸显示屏, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34寸显示器, $499, 缺货, 预计下周到货',
    }
    
    key = product_name.lower().strip()
    if key in product_catalog:
        return f"【产品数据】: {product_catalog[key]}"
    else:
        # 模糊匹配提示
        available = ", ".join(product_catalog.keys())
        return f"抱歉，没有找到 '{product_name}' 的信息。现有产品: {available}"

# 3. 创建 Agent
llm = ChatOpenAI(
    model=model_name,
    api_key=api_key,
    base_url=base_url,
    model_kwargs={"extra_body": extra_body}
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个外部供应商的产品目录专家 Agent。
    当被问及产品时，必须使用 get_product_info 工具查询数据库。
    如果被问及多个产品，请分别查询。
    请提供清晰、准确的信息，包括价格和库存。
    """),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, [get_product_info], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[get_product_info], verbose=True)

# 定义输入 Schema 以修复 500 错误
class Input(BaseModel):
    input: str = Field(description="User input question")

# 增强链：提取 output
chain = agent_executor | RunnableLambda(lambda x: x["output"])

# 4. 创建 Web 服务 (LangServe)
app = FastAPI(
    title="Product Catalog Agent Server",
    version="1.0",
    description="一个提供产品查询服务的远程 Agent"
)

# 添加路由 /product_catalog
add_routes(
    app,
    chain.with_types(input_type=Input),
    path="/product_catalog",
)

if __name__ == "__main__":
    print("🚀 正在启动 Product Catalog Server (端口 8001)...")
    uvicorn.run(app, host="localhost", port=8001)
'''

# 将代码写入文件
with open("product_catalog_server.py", "w", encoding="utf-8") as f:
    f.write(server_code)

print("📝 服务端代码已写入 `product_catalog_server.py`")

📝 服务端代码已写入 `product_catalog_server.py`


### 1.2 启动后台服务

现在我们使用 `subprocess` 在后台启动这个服务。这模拟了一个完全独立的、可能由第三方维护的远程服务。

In [4]:
# 在后台启动 Server
process = subprocess.Popen(
    ["python", "product_catalog_server.py"], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE,
    env={**os.environ},  # 传递当前环境变量 (包含 API Key)
    text=True  # 文本模式
)

print("🚀 Server 进程已启动，正在等待服务就绪...")

# 健康检查循环
server_url = "http://localhost:8001/product_catalog"
max_retries = 30
for i in range(max_retries):
    try:
        # LangServe 会自动生成 /input_schema 端点，我们可以用它来检查服务是否存活
        resp = requests.get(f"{server_url}/input_schema", timeout=1)
        if resp.status_code == 200:
            print(f"✅ Server 已就绪: {server_url}")
            break
    except Exception:
        time.sleep(1)
        print(".", end="", flush=True)
else:
    print("\n❌ Server 启动超时。请检查日志或端口占用情况。")
    # 打印一些日志以供调试
    if process.poll() is not None:
        stdout, stderr = process.communicate()
        print(f"Server Output: {stderr}")

🚀 Server 进程已启动，正在等待服务就绪...
...✅ Server 已就绪: http://localhost:8001/product_catalog


## 🎧 Section 2: 创建 Customer Support Agent (客户端)

既然“供应商”的目录服务已经在 8001 端口运行，我们现在的角色切换为“客户支持团队”。我们需要构建一个 Agent 来回答客户问题，并在需要时调用那个远程服务。

### 2.1 定义远程连接工具

我们使用 `RemoteRunnable` 连接到远程服务，并将其封装为一个本地 Tool。

In [5]:
# 1. 连接到远程 Agent
remote_agent = RemoteRunnable("http://localhost:8001/product_catalog")

# 2. 将远程 Agent 封装为工具
@tool
def call_product_catalog(query: str) -> str:
    """ 
    Useful when you need to find product information, prices, stock, or specifications.
    The query should be a clear question about products.
    Example: "How much is the iPhone 15?" or "Do you have Dell XPS in stock?"
    """
    # 调用远程 Agent
    # LangServe 的 invoke 接受字典输入
    print(f"    📡 [A2A Call] 正在呼叫远程 Product Catalog Agent: '{query}'")
    try:
        result = remote_agent.invoke({"input": query})
        # 由于服务端已简化为只返回 output 字符串，这里 result 就是最终回答
        return str(result)
    except Exception as e:
        return f"Error calling remote agent: {e}"

print("✅ 远程工具 `call_product_catalog` 已就绪")

✅ 远程工具 `call_product_catalog` 已就绪


### 2.2 构建 Customer Support Agent

这个 Agent 运行在本地（本 Notebook 中），它的任务是处理用户请求。当遇到产品问题时，它会使用 `call_product_catalog` 工具，实际上就是发起了 A2A 通信。

In [6]:
client_llm = ChatOpenAI(
    model=config["model_name"],
    api_key=config["api_key"],
    base_url=config["base_url"],
    model_kwargs={"extra_body": config["extra_body"]}
)

client_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个友好、专业的客户支持 Agent。
    当客户询问产品信息时：
    1. 必须使用 `call_product_catalog` 工具查询详细信息。
    2. 不要编造价格或库存信息。
    3. 在获得远程 Agent 的回复后，用亲切的语气将其转述给客户。
    """),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

support_agent = create_tool_calling_agent(client_llm, [call_product_catalog], client_prompt)
support_runner = AgentExecutor(agent=support_agent, tools=[call_product_catalog], verbose=True)

print("✅ Customer Support Agent 已构建完成")

✅ Customer Support Agent 已构建完成


## 🧪 Section 3: 测试 A2A 通信

现在让我们来测试一下！

流程如下：
1. **用户**向 **Support Agent** 提问。
2. **Support Agent** 思考并决定调用 `call_product_catalog`。
3. **Local Tool** 发送 HTTP 请求给 `http://localhost:8001`。
4. **Product Catalog Agent** (Server) 接收请求，查询模拟数据库，返回结果。
5. **Support Agent** 收到结果，组织语言回答用户。

In [7]:
# 测试案例 1: 查询单个产品
query1 = "请问 iPhone 15 Pro 有货吗？价格是多少？"
print(f"👤 Customer: {query1}\n")

response1 = support_runner.invoke({"input": query1})

print(f"\n🎧 Support Agent: {response1['output']}")

👤 Customer: 请问 iPhone 15 Pro 有货吗？价格是多少？



> Entering new AgentExecutor chain...

Invoking: `call_product_catalog` with `{'query': 'iPhone 15 Pro 有货吗？价格是多少？'}`


    📡 [A2A Call] 正在呼叫远程 Product Catalog Agent: 'iPhone 15 Pro 有货吗？价格是多少？'
iPhone 15 Pro 当前有货，但库存紧张，仅剩8台。  
价格为 $999，配备128GB 存储容量，采用钛金属设计。您好！iPhone 15 Pro 目前有货，但库存比较紧张，只剩下8台了哦～  
价格是 $999，配备128GB 存储空间，采用轻盈坚固的钛金属设计。  
如果您有兴趣，建议尽快下单，以免错过呢！😊

> Finished chain.

🎧 Support Agent: 您好！iPhone 15 Pro 目前有货，但库存比较紧张，只剩下8台了哦～  
价格是 $999，配备128GB 存储空间，采用轻盈坚固的钛金属设计。  
如果您有兴趣，建议尽快下单，以免错过呢！😊


In [8]:
# 测试案例 2: 产品比较 (需要多次查询或Agent自行处理)
query2 = "我想买个笔记本电脑，对比一下 Dell XPS 15 和 Macbook Pro 14 的价格和配置"
print(f"👤 Customer: {query2}\n")

response2 = support_runner.invoke({"input": query2})

print(f"\n🎧 Support Agent: {response2['output']}")

👤 Customer: 我想买个笔记本电脑，对比一下 Dell XPS 15 和 Macbook Pro 14 的价格和配置



> Entering new AgentExecutor chain...

Invoking: `call_product_catalog` with `{'query': '对比 Dell XPS 15 和 Macbook Pro 14 的价格和配置'}`


    📡 [A2A Call] 正在呼叫远程 Product Catalog Agent: '对比 Dell XPS 15 和 Macbook Pro 14 的价格和配置'
以下是 Dell XPS 15 和 MacBook Pro 14 的价格与配置对比：

| 项目 | Dell XPS 15 | MacBook Pro 14 |
|------|-------------|---------------|
| **价格** | $1,299 | $1,999 |
| **库存** | 45台 | 22台 |
| **显示屏** | 15.6寸 | 未明确尺寸（标准为14.2寸） |
| **处理器** | 未提供具体型号（通常为Intel Core i7/i9） | Apple M3 Pro芯片 |
| **内存** | 16GB RAM | 18GB RAM |
| **存储** | 512GB SSD | 512GB SSD |

### 简要分析：
- **价格方面**：Dell XPS 15 比 MacBook Pro 14 便宜 $700，性价比更高。
- **性能方面**：MacBook Pro 搭载 M3 Pro 芯片，在能效和专业工作负载（如视频剪辑、开发）中表现更优；XPS 15 使用 Intel 处理器，兼容性广泛，适合 Windows 生态用户。
- **内存**：MacBook Pro 提供 18GB RAM，略高于 XPS 15 的 16GB。
- **库存情况**：XPS 15 库存更充足（45台 vs 22台），更容易购买。

✅ 如果你偏好 Windows 系统、需要大屏和较高性价比，**Dell XPS 15** 是不错选择。  
✅ 如果你注重系统流畅性、续航以及苹果生态整合，愿意为性能和设计多付费，推荐 **MacBook Pro

In [9]:
# 🧹 清理工作: 关闭后台服务器
print("正在关闭 Product Catalog Server...")
process.terminate()
try:
    process.wait(timeout=5)
    print("✅ Server 已关闭。")
except subprocess.TimeoutExpired:
    process.kill()
    print("⚠️ Server 强制关闭。")

正在关闭 Product Catalog Server...
✅ Server 已关闭。


## 📊 总结

在这个简单的示例中，我们实现了一个完整的 **Agent2Agent** 架构：

| 角色 | 实现技术 | 说明 |
| :--- | :--- | :--- |
| **服务端 (Vendor)** | LangServe + FastAPI | 将 Agent 暴露为标准 HTTP API |
| **通信协议** | HTTP / JSON | 跨网络、跨语言的标准交互方式 |
| **客户端 (Consumer)** | RemoteRunnable + Tool | 像调用本地函数一样调用远程智能体 |

这种模式非常强大，它允许我们将复杂的系统拆分为多个专门的 Micro-Agents，每个 Agent 都可以由不同的团队维护，甚至使用不同的编程语言编写，只要它们遵循相同的 API 标准。